# Dane (eda.ipynb)
Dane o wypadkach (https://geoportal.wroclaw.pl/zasoby/?zasob=wypadki_kolizje)

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
# gdf = gpd.read_file("./data/wypadki/2023_szczegoly_zdarzen_dr.shx")
gdf = gpd.read_file("./2023_szczegoly_zdarzen_dr.shx")

In [ ]:
gdf

# Mapa ciepła
Na podstawie (https://www.kaggle.com/code/holoong9291/heatmap-for-car-accident/notebook)

**ZADANIE**: <br>
Załóżmy, że chcemy wyświetlić na mapie miejsca wypadków wraz z liczbą uczestników biorących w nich udział w 2023 roku.

Pierwszym ze sposobów będzie wykorzystanie `.explore`, dzięki czemu będziemy mogli wyświetlić punkty na mapie. Dodatkowo ustawiając argumenty `column="UCZ_SUMA"` oraz `cmap="viridis"` będziemy mogli zobaczyć liczbę uczestników danego zdarzenia (odcień punktu mówi nam ilu było uczestników).

In [ ]:
pip install mapclassify

In [ ]:
gdf2 = gdf.copy()
gdf2.explore(column="UCZ_SUMA", cmap="viridis")

W powyższym sposobie niezależnie czy przybliżamy, czy oddalamy mapę, zawsze będziemy widzieć wszystkie miejsca wypadków (punkty) z osobna.

Powyższe zadanie możemy wykonać również w inny sposób. Wykorzystując bibliotekę `folium`, dzięki `HeatMap` możemy podobnie jak wcześniej zaznaczyć miejsca wypadków, z tą różnicą, że jeżeli mamy bardziej oddaloną mapę te miejsca będziemy widzieć bardziej ogólnie. Z kolei im bardziej będziemy przybliżać dany obszar, tym precyzyjniej będziemy widzieć dokładne miejsce zdarzenia. Dodatkowo kolory będą nam sygnalizowały jak dużo uczestników było na danym obszarze.

In [ ]:
import folium
from folium.plugins import HeatMap

Współrzędne naszych danych są zapisane w układzie współrzędnych EPSG: 2177 (patrz poniżej).  

In [ ]:
gdf2['geometry']

Aby móc utworzyc mapę ciepła za pomocą biblioteki `folium` musimy przetransformować współrzędne punktów do układu współrzędnych EPSG: 4326.

In [ ]:
gdf3 = gdf2.copy()
gdf3 = gdf3.to_crs('EPSG:4326')

Teraz mamy:

In [ ]:
gdf3['geometry']

In [ ]:
X = gdf3['geometry'].x # współrzędne punktów na osi x
Y = gdf3['geometry'].y # współrzędne punktów na osi y
UCZ_SUMA = gdf3['UCZ_SUMA'] # liczba uczestników

tmp = list(zip(Y, X, UCZ_SUMA)) # tworzenie listy współrzędnych dla mapy ciepła

hmap = folium.Map(location=[Y.mean(), X.mean()], control_scale=True, zoom_start=10) # tworzenie Folium map

hmap.add_child(HeatMap(tmp, radius=15)) # dodanie warstwy mapy ciepła

hmap

# Regionalizacja, embedding
W tej części będziemy wykorzystywać bibliotekę `srai` (https://github.com/kraina-ai/srai)

In [ ]:
pip install srai

In [ ]:
pip install srai[osm]

In [ ]:
from srai.regionalizers import H3Regionalizer, geocode_to_region_gdf

In [ ]:
from srai.loaders.osm_loaders.filters.popular import get_popular_tags
from srai.loaders.osm_loaders.filters import GEOFABRIK_LAYERS, HEX2VEC_FILTER
from srai.loaders.osm_loaders import OSMOnlineLoader
from srai.regionalizers import geocode_to_region_gdf
from srai.plotting.folium_wrapper import plot_regions
from functional import seq

## Regionalizacja
Załóżmy, że chcemy podzielić Wrocław na heksagony.

In [ ]:
area = geocode_to_region_gdf("Wrocław, Poland") # pobieranie obszaru

In [ ]:
area

Możliwe regionalizacje (https://github.com/kraina-ai/srai#regionalization)

In [ ]:
regionalizer = H3Regionalizer(resolution=8) # https://h3geo.org/docs/core-library/restable/
regions = regionalizer.transform(area)

In [ ]:
regions.head()

In [ ]:
from srai.plotting import plot_regions

In [ ]:
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")

In [ ]:
folium_map # wyświetlenie pobranego obszaru

In [ ]:
plot_regions(regions_gdf=regions, map=folium_map) # wyświetlenie obszaru wraz z regionzalizacją

## Embedding
**ZADANIE:**<br>
Załóżmy, że chcemy zobaczyć, jak rozkładała się liczba wypadków w poszczególnych regionach w 2023 roku.

In [ ]:
pip install typing-extensions

In [ ]:
from srai.embedders import CountEmbedder
from srai.joiners import IntersectionJoiner
from srai.loaders import OSMOnlineLoader

In [ ]:
loader = OSMOnlineLoader()
regionalizer = H3Regionalizer(resolution=9)
joiner = IntersectionJoiner()

In [ ]:
from srai.constants import WGS84_CRS, FEATURES_INDEX

In [ ]:
gdf3.head()

In [ ]:
area = geocode_to_region_gdf("Wrocław, Poland") # pobieranie obszaru

query = {'ROK': gdf3['ROK'], 'geometry': gdf3['geometry']} # wyznaczanie jakie dane będziemy chcięli wyciągnąć
features = gpd.GeoDataFrame(
    query,
    crs=WGS84_CRS,
    index=gpd.pd.Index(name=FEATURES_INDEX, data=[i for i in range(len(gdf3))]),
)

In [ ]:
regions = regionalizer.transform(area)

In [ ]:
features.head()

In [ ]:
joint = joiner.transform(regions, features, return_geom=True) # łączenie regionów z punktami w których były wypadki

`joint` zwraca tabelę, z której można odczytać, które punkty leżą w danym regionie.

In [ ]:
joint

Mapa wraz z regionalizacją + punkty w których doszło do wypadków na podstawie pobranych danych <br>
(można zauważyć, że część punktów jest poza obszarem Wrocławia):

In [ ]:
folium_map = plot_regions(regions, colormap=["cornflowerblue"])
features.explore(
    m=folium_map,
    style_kwds=dict(color="darkblue", opacity=0.5, fillColor="darkblue", fillOpacity=0.5),
    marker_kwds=dict(radius=2),
)

 Mapa wraz z regionalizacją + punkty w których doszło do wypadków już po sprawdzeniu które punkty należą do regionów <br>
(można zauważyć, że każdy z punktów mieści się w którymś z regionów, nie ma punktów poza regionami):

In [ ]:
folium_map = plot_regions(regions, colormap=["cornflowerblue"])
joint.explore(
    m=folium_map,
    style_kwds=dict(color="darkgreen", opacity=1, fillColor="darkgreen", fillOpacity=1),
    marker_kwds=dict(radius=2),
)

Wyświtlenie dwóch powyższych map z punktami na jednej mapie:

In [ ]:
folium_map = plot_regions(regions, colormap=["cornflowerblue"])

features.explore(
    m=folium_map,
    style_kwds=dict(color="darkblue", opacity=0.5, fillColor="darkblue", fillOpacity=0.5),
    marker_kwds=dict(radius=2),
)

joint.explore(
    m=folium_map,
    style_kwds=dict(color="darkgreen", opacity=1, fillColor="darkgreen", fillOpacity=1),
    marker_kwds=dict(radius=2),
)

Mapa wraz z regionalizacją + punkty wypadków (kolory oznaczają regiony, jeżeli dane punkty znajdują się w jednym regionie to są tego samego koloru):

In [ ]:
from srai.constants import REGIONS_INDEX
from plotly.express import colors

In [ ]:
folium_map = plot_regions(regions, tiles_style="CartoDB positron", colormap=["rgba(0,0,0,0)"])
joint.reset_index().explore(m=folium_map, column=REGIONS_INDEX, cmap=colors.qualitative.Bold)

W poniższej tabeli `ROK_2023` będzie oznaczać liczę wpadków w 2023 w danym regionie:

In [ ]:
wide_embedder = CountEmbedder(count_subcategories=True)
wide_embedding = wide_embedder.transform(regions, features, joint)
wide_embedding.head()

Liczba regionów z daną liczbą wypadków:

In [ ]:
import matplotlib.pyplot as plt

x=[i for i in range(0, max(wide_embedding["ROK_2023"])+1)]
dict_x_y={}
for i in x:
  dict_x_y[i] = list(wide_embedding["ROK_2023"]).count(i)

y=list(dict_x_y.values())

plt.grid(axis='y', alpha=0.75)
plt.bar(x,y, color='darkorange')
plt.title('Liczba regionów z daną liczbą wypadków')
plt.xlabel('Liczba wypadków')
plt.ylabel('Liczba regionów')
plt.show()

Wyświetlenie regionów na mapie w różnych odcieniach w zależności od liczby wypadków:

In [ ]:
from srai.plotting import plot_numeric_data

In [ ]:
plot_numeric_data(regions, "ROK_2023", wide_embedding)

# Skrzyżowania i sygnalizacja świetlna we Wrocławiu

Tutaj już nie będziemy wykorzystywać danych o wypadkach. Załóżmy, że chcemy się dowiedzieć, gdzie są zlokalizowane skrzyżowania i sygnalizacja świetlna.
<br>
<br>
Możemy to zrobić na dwa sposob:
* wykorzystać bibliotekę `srai` (szybszy sposób),
* wykorzystać bibliotekę `overpy` (bardziej skomplikowane) - ten sposób zostanie przedstawiony tylko w sekcji 'Skrzyżowania'.
<br>
<br>


## Skrzyżowania

**ZADANIE:**<br>
Załóżmy, że chcemy pobrać dane o skrzyżowaniach we Wrocławiu, konkretnie chcemy wiedzieć w jakich punktach się mieszczą.

Zacznijmy od sposobu z wykorzystaniem biblioteki `overpy`.

Biblioteka `overpy`, jest wrapperem do API serwisu OpenStreetMap (OSM), służy do pobierania danych geograficznych. Za pomocą `overpy` można komunikować się z API OSM i pobierać różnego rodzaju informacje geograficzne, takie jak dane o miejscach, drogach, budynkach itp., które są dostępne na platformie OpenStreetMap.

In [ ]:
pip install overpy

In [ ]:
import overpy

api = overpy.Overpass()

query = """
area[name="Wrocław"];
(node["highway"="crossing"](area););
out;
""" # zapytanie o skrzyżowania we Wrocławiu

result = api.query(query)

In [ ]:
data = []
for node in result.nodes:
    data.append({
        'ID': node.id,
        'Latitude': node.lat,
        'Longitude': node.lon,
    })

In [ ]:
from shapely.geometry import Point

# tworzenie obiektów Point na podstawie współrzędnych
geometry = []
for i in data:
  geometry.append(Point(i['Longitude'], i['Latitude']))

gdf_overpy = gpd.GeoDataFrame(data, geometry=geometry) # tworzenie GeoDataFrame

gdf_overpy.head()

Drugim sposobem jest wykorzystanie biblioteki `srai`.

In [ ]:
loader = OSMOnlineLoader()

query = {"highway": "crossing"} # filtr dla skrzyżowań
area = geocode_to_region_gdf("Wrocław, Poland") # uzyskanie obszaru Wrocławia jako GeoDataFrame
features = loader.load(area, query) # wczytanie danych o skrzyżowaniach we Wrocławiu

features.head()

Porównując powyższe tabele widzimy, że w obu sposobach w rezultacie otrzymujemy te same punkty.

Od teraz będziemy parcować tylko na danych uzyskanch za pomocą biblioteki `srai`.

Teraz wyświetlimy sobie miejsca na mapie gdzie występują skrzyżwania:

In [ ]:
folium_map = plot_regions(area, colormap=["lightgray"], tiles_style="CartoDB positron")
features.explore(m=folium_map, color="gold")

Teraz dla danych o skrzyżowaniach, podobnie jak poprzednio, zostanie dokonana regionalizacja i embedding  (patrz sekcja 'Regionalizacja, Embedding').

Będziemy zliczać liczbę skrzyżowań w danym regionie.

In [ ]:
# -----KOD Z POPRZEDNIEJ KOMÓRKI-----
# loader = OSMOnlineLoader()

# query = {"highway": "crossing"} # filtr dla skrzyżowań
# area = geocode_to_region_gdf("Wrocław, Poland") # uzyskanie obszaru Wrocławia jako GeoDataFrame
# gdf_srai_crossing = loader.load(area, query) # wczytanie danych o skrzyżowaniach we Wrocławiu
# ----------------------------------

regionalizer = H3Regionalizer(resolution=9)
joiner = IntersectionJoiner()

regions = regionalizer.transform(area)
joint = joiner.transform(regions, features)
embedder = CountEmbedder()
embeddings = embedder.transform(regions, features, joint)

Liczba regionów z daną liczbą skrzyżowań:

In [ ]:
x=[i for i in range(0, max(embeddings["highway_crossing"])+1)]
dict_x_y={}
for i in x:
  dict_x_y[i] = list(embeddings["highway_crossing"]).count(i)

y=list(dict_x_y.values())

plt.grid(axis='y', alpha=0.75)
plt.bar(x,y, color='darkorange')
plt.title('Liczba regionów z daną liczbą skrzyżowań')
plt.xlabel('Liczba skrzyżowań')
plt.ylabel('Liczba regionów')
plt.show()

Wyświetlenie regionów na mapie w różnych odcieniach w zależności od liczby skrzyżowań:

In [ ]:
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions, "highway_crossing", embeddings, map=folium_map)

Mapa wraz z regionalizacją + pukty, gdzie są skrzyżowania:

In [ ]:
folium_map = plot_regions(regions, tiles_style="CartoDB positron", colormap=["lightgray"])
features.explore(
    m=folium_map,
    color="gold"
)

## Sygnalizacja świetlna

**ZADANIE:**<br>
Podobnie jak dla skrzyzowań, chcemy teraz pobrać dane o sygnalizacji świetlnej we Wrocławiu.

W tym zadaniu postępujemy analogicznie jak dla skrzyżowań (korztystamy z biblioteki `srai`).

In [ ]:
query = {"highway": "traffic_signals"}
area = geocode_to_region_gdf("Wrocław, Poland")
features = loader.load(area, query)
regions = regionalizer.transform(area)

folium_map = plot_regions(regions, tiles_style="CartoDB positron", colormap=["lightgray"])
features.explore(
    m=folium_map,
    color="purple"
)

In [ ]:
features.head()

In [ ]:
joint = joiner.transform(regions, features)

embedder = CountEmbedder()
embeddings = embedder.transform(regions, features, joint)

folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions, "highway_traffic_signals", embeddings, map=folium_map)

In [ ]:
x=[i for i in range(0, max(embeddings["highway_traffic_signals"])+1)]
dict_x_y={}
for i in x:
  dict_x_y[i] = list(embeddings["highway_traffic_signals"]).count(i)

y=list(dict_x_y.values())

plt.grid(axis='y', alpha=0.75)
plt.bar(x,y, color='darkorange')
plt.title('Liczba regionów z daną liczbą sygnalizacji świetlnej')
plt.xlabel('Liczba sygnalizacji świetlnej')
plt.ylabel('Liczba regionów')
plt.show()

## Sygnalizacja świtlna i skrzyżowania

### Filtrowanie
Nie trzeba pobierać danych każde z osobna, możemy również tworzyć filtry dla większej ilości tagów.


**ZADANIE:**<br>
Załóżmy, że chcemy pobrać równocześnie dane o skrzyżowaniach i sygnalizacji.

In [ ]:
query = {"traffic_signals": {"highway": "traffic_signals"}, "crossing": {"highway": "crossing"}} # filtr dla sygnalizacji świetlnej i skrzyżowań
area = geocode_to_region_gdf("Wrocław, Poland") # uzyskanie obszaru Wrocławia jako GeoDataFrame
features = loader.load(area, query) # wczytanie danych o sygnalizacji świetlnej i skrzyżowaniach we Wrocławiu

# regionalizacja danych
regionalizer = H3Regionalizer(resolution=9)
regions = regionalizer.transform(area)

In [ ]:
features.head()

### Embedding

In [ ]:
# Łączenie danych
joiner = IntersectionJoiner()
joint = joiner.transform(regions, features)

# Embedding danych
embedder = CountEmbedder()
embeddings = embedder.transform(regions, features, joint)

# Tworzenie mapy Folium
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")


In [ ]:
embeddings.head()

In [ ]:
plot_numeric_data(regions, "traffic_signals_highway=traffic_signals", embeddings, map=folium_map)

In [ ]:
plot_numeric_data(regions, "crossing_highway=crossing", embeddings, map=folium_map)